<a href="https://colab.research.google.com/github/CUN-bjy/AI502/blob/main/%5BAI502%5D_Programming_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm.auto import tqdm

In [56]:
# Set the device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [57]:
# Define the FCN model
class FCN(nn.Module):
    def __init__(self, in_dim = 3072, out_dim = 10, num_layer=3):
        super(FCN, self).__init__()
        """
        Fully Connected Network
        """
        self.num_layer = num_layer
        self.layer1 = nn.Linear(in_dim, 512)
        self.layer2 = nn.Linear(512, 128)
        self.layer3 = nn.Linear(128, out_dim)

    def forward(self, x):
        """
        Forward function of FCN
        """
        x = x.view(x.size(0), -1)
        x = nn.functional.sigmoid(self.layer1(x))
        x = nn.functional.sigmoid(self.layer2(x))
        out = nn.functional.sigmoid(self.layer3(x))

        return out

In [58]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, out_channels=10, kernel_size=3, stride=1, padding=0, active_fn=nn.functional.sigmoid):
        super(CNN, self).__init__()
        """
        Convolutional Neural Network
        """  
        self.active_fn = active_fn

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=kernel_size, stride=stride, padding=padding)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        
        in_dim = 32
        for i in range(2): # num_layer
          in_dim = int((in_dim + 2*padding - (kernel_size-1) - 1) / stride + 1)

        self.fcn = FCN(out_channels * in_dim * in_dim)


    def forward(self, x):
        """
        Forward function of CNN
        """
        batch_size = x.size(0)
        x = self.active_fn(self.conv1(x))
        conv_out = self.active_fn(self.conv2(x))

        conv_out.view(batch_size, -1)
        out = self.fcn(conv_out)

        return out

In [59]:
# Set the hyperparameters
"""You can change those values"""
learning_rate = 0.001
num_epochs = 100
batch_size = 128
weight_decay = 0

In [60]:
# Initialize the model and optimizer
# model = FCN().to(device)
model = CNN().to(device)

"""You can change the optimizer"""
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [61]:
# CIFAR preprocessing 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [62]:
train_losses = []
test_accs = []
for epoch in tqdm(range(num_epochs)):
    # Training
    epoch_loss = 0.0
    for images, labels in train_loader:
        # Move the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Evaluation
    if (epoch + 1) % 10 == 0:
        num_correct = 0
        num_total = 0
        for images, labels in test_loader:
            # Move the images and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            num_correct += torch.sum(torch.argmax(outputs, dim=-1) == labels)
            num_total += len(labels)
        test_accs.append(100 * num_correct / num_total)
        print('[epoch %d] test accuracy: %.2f' %
            (epoch + 1, 100 * num_correct / num_total))

    # Print statistics
    train_losses.append(epoch_loss / len(train_loader))
    print('[epoch %d] training loss: %.4f' %
            (epoch + 1, epoch_loss / len(train_loader)))
    
print('Finished Training')

  0%|          | 0/100 [00:00<?, ?it/s]

[epoch 1] training loss: 2.3041


KeyboardInterrupt: ignored